In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from prompts.prompt_04 import base_prompt,system_prompt,mimic_prompt,init_prompt

load_dotenv()  

# Configurazione dell'API
XAI_API_KEY = os.getenv("XAI_API_KEY")

client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)